In [1]:
import torch
from torch.autograd.functional import jacobian
sys.path.append("../../models/")
from CustomCNNVessel import CustomResNet

def get_all_gradients(model, image, sampling_rate=10, device="cuda", vectorize=False):
  torch.cuda.empty_cache()

  model.to(device)
  model = model.eval()
  image = image.to(device).unsqueeze(0)
  sampled_image = image[:,:,::sampling_rate,::sampling_rate].to(device)
  sampled_image.requires_grad = True
  
  jacobian_gradient = torch.autograd.functional.jacobian(model, 
                                                         sampled_image,
                                                         vectorize = vectorize)
  jacobian_gradient = jacobian_gradient.squeeze().to('cpu')
  return jacobian_gradient

def wrapper(model, first_row, last_row):
    '''Wrap model to return only rows in range [first_row, last_row]'''
    def new_model(img):
        out = model(img)
        return out[:,:,first_row:last_row]
    return new_model

def get_all_gradients_chunks(model, image, rows_proc=28, device="cuda"):

  c, nr, nc = image.shape
  model.eval()
  model.to(device)

  size_jac = (2, nr, nc, nr, nc)
  # Number of image chunks to process
  nchunks = nr//rows_proc
  nchunks += nr%rows_proc!=0   # Add 1 if shape is not divisible by rows_proc
  image_cuda = image.unsqueeze(0).to(device).requires_grad_()

  jacobian_gradient = torch.zeros(*size_jac, device='cpu')
  first_row = 0
  for idx in range(nchunks):
      print(idx)
      last_row = first_row + rows_proc
      model_w = wrapper(model, first_row, last_row)
      out = jacobian(model_w, image_cuda, vectorize=False).to('cpu')  
      # Cannot use .squeeze here since one valid size of out might be 1
      jacobian_gradient[:,first_row:last_row] = out[0,:,:,:,0,0]
      first_row = last_row

  return jacobian_gradient

def compare(model, image, rows_proc):
  '''Compare methods'''
   
  torch.backends.cudnn.deterministic = True
  ref = get_all_gradients(model, image, sampling_rate=1)
  jacobian_gradient = get_all_gradients_chunks(model, image, rows_proc)
  print(torch.allclose(ref, jacobian_gradient))

model = CustomResNet(num_classes=2)
rows_proc = 4   # Process image rows_proc at a time
image = torch.rand(1, 224, 224)[:,:10,:10]

jacobian_gradient = get_all_gradients_chunks(model, image, rows_proc)

ModuleNotFoundError: No module named 'CustomCNNVessel'

In [ ]:
def get_gradients_in_batches(model, image, batch_size, device="cpu"):
    c, h, w = image.shape
    # Initialize a tensor to hold the full gradient
    full_gradient = torch.zeros((c, h, w), device='cpu')

    for i in range(0, h, batch_size):
        for j in range(0, w, batch_size):
            # Calculate the size of the current batch
            current_batch_size_h = min(batch_size, h - i)
            current_batch_size_w = min(batch_size, w - j)

            # Extract a batch from the image
            batch = image[:, i:i + current_batch_size_h, j:j + current_batch_size_w].unsqueeze(0).to(device)
            batch.requires_grad = True

            # Compute the gradient for the batch
            output = model(batch)
            output.backward(torch.ones_like(output))
            batch_gradient = batch.grad.squeeze().cpu()

            # Assign the batch gradient to the corresponding region in the full gradient tensor
            full_gradient[:, i:i + current_batch_size_h, j:j + current_batch_size_w] = batch_gradient

            # Free memory
            del batch, output, batch_gradient
            if device == "cuda":
                torch.cuda.empty_cache()  # Release GPU memory

    return full_gradient


In [7]:
compare(model, image, rows_proc)

0
1
2
True
